In [1]:
import json
from openai import OpenAI

In [2]:
client = OpenAI()

In [3]:
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are an assistant that creates a slogan based on company description."},
        {"role": "user", "content": "A company that sells ice cream."},
    ],
    n=1,
    temperature=1
)

In [4]:
completion_json = json.loads(completion.json())

In [5]:
print(completion_json)

{'id': 'chatcmpl-9GolAjbLO38asbuQ5KfvsQjATWiXg', 'choices': [{'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': '"Scooping up happiness one cone at a time!"', 'role': 'assistant', 'function_call': None, 'tool_calls': None}}], 'created': 1713795396, 'model': 'gpt-3.5-turbo-0125', 'object': 'chat.completion', 'system_fingerprint': 'fp_c2295e73ad', 'usage': {'completion_tokens': 11, 'prompt_tokens': 31, 'total_tokens': 42}}


In [6]:
completion_json

{'id': 'chatcmpl-9GolAjbLO38asbuQ5KfvsQjATWiXg',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': '"Scooping up happiness one cone at a time!"',
    'role': 'assistant',
    'function_call': None,
    'tool_calls': None}}],
 'created': 1713795396,
 'model': 'gpt-3.5-turbo-0125',
 'object': 'chat.completion',
 'system_fingerprint': 'fp_c2295e73ad',
 'usage': {'completion_tokens': 11, 'prompt_tokens': 31, 'total_tokens': 42}}

In [7]:
print(completion_json['choices'][0]['message']['content'])

"Scooping up happiness one cone at a time!"


# Embeddings

In [8]:
import numpy as np

from openai import OpenAI

In [9]:
def create_embeddings(text):
    embedding = client.embeddings.create(input=text, model="text-embedding-ada-002").data[0].embedding

    return embedding

In [10]:
def compare_two_embeddings(a, b):
    cos_sim = np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

    return cos_sim

In [11]:
text_1 = 'I like apples'
text_2 = 'I like bananas'

In [12]:
round(compare_two_embeddings(create_embeddings(text_1), create_embeddings(text_2)), 2)

0.9

In [13]:
text_1 = "I like apples" 
text_2 = "All legislative Powers herein granted shall be vested in a Congress of the United States, which shall consist of a Senate and House of Representatives."

In [14]:
round(compare_two_embeddings(create_embeddings(text_1), create_embeddings(text_2)), 2)

0.71

# Fine-Tuning

In [15]:
import os 
from openai import OpenAI

In [16]:
training_data = client.files.create(
    file = open('chapter4trainingdata.json', 'rb'),
    purpose = 'fine-tune'
)

file_id = training_data.id

In [17]:
fine_tune_job = client.fine_tuning.jobs.create(training_file=file_id,
                                               model='gpt-3.5-turbo')

In [ ]:
client.fine_tuning.jobs.retrieve(fine_tune_job.id)

FineTuningJob(id='ftjob-X0CK8OnAadmbxPriN81Btuln', created_at=1713523591, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=10, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-REx38PW3oxp2zqN0gOBv9tII', result_files=[], seed=7357403, status='running', trained_tokens=None, training_file='file-qsswzZGLpUQQLIL2arzjyiVJ', validation_file=None, integrations=[], user_provided_suffix=None)

In [ ]:
import time
status = 'running'

while status != 'succeeded':

    print(' '*20, end='\r')
    print(status, end='\r')
    time.sleep(0.4) 
    print(' '*20, end='\r')
    print(status + '.', end='\r')
    time.sleep(0.4) 
    print(' '*20, end='\r')
    print(status + '..', end='\r')
    time.sleep(0.4) 
    print(' '*20, end='\r')
    print(status + '...', end='\r')
    time.sleep(0.4) 

    # Verifica cual es el status del job
    status = client.fine_tuning.jobs.retrieve(fine_tune_job.id).status

print(status)
    

succeeded.          


In [ ]:
fine_tuned_model = client.fine_tuning.jobs.retrieve(fine_tune_job.id).fine_tuned_model

In [96]:
completion = client.chat.completions.create(
    model=fine_tuned_model,
    messages=[
        {"role": "system", "content": "You are an assistant that creates a funnny one-line jokes based on a given scenario."},
        {"role": "user", "content": "A man walkin across the road."},
    ]
)

In [99]:
print(completion.choices[0].message.content)

Why did the man bring a ladder to the crosswalk? Because he heard the pedestrian 'crossing' was a promotion! Haha
